# Mishra Data Quality Index 4 (p4g.py)
Max Calzada 
January 6, 2023 (Latest Update)

'dqi4_4sentences.csv' or NLP sentences to be analyzed must be manually inserted into the content folder each session.

To analyze NLP data, command `gwords=pd.read_csv("dqi4_4sentences.csv")` must be edited so that the argument in the `pd.read_csv()` command matches the name of the data set.

## What does DQI c4 do?

DQI c4 measures intra-sample word similarity, or the word similarity within each sentence or datum.

The closer the average similarity score is towards the minimum value, the higher the data quality.

## Appropriate Data Format

To read in NLP data, make sure that the data is in a .csv format. The data should only have one column titled `documents`. Each cell should contain one datum.

In [1]:
import pandas as pd
import numpy as np 
import json
import csv

import matplotlib.pyplot as plt
# import seaborn as sns

from tqdm import tqdm # issue with tqdm?
from tqdm import tqdm_notebook as tqdm # needed? # correct?
# Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
# nltk.download('averaged_perceptron_tagger')

import spacy
import math

import string
import sys
import random
import pickle


from collections import Counter
from itertools import chain

nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

import spacy.cli

spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg") # spacy.load("en_trf_bertbaseuncased_lg") #

from sklearn.metrics.pairwise import cosine_similarity

from spacy.lang.en.examples import sentences # needed?

from numba import jit, cuda

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
gwords=pd.read_csv("dqi4_4sentences.csv") # pd.read_csv("path_to_dev_good_words/dev_good_words.csv")

@jit
def parameter4(sentence,size,WSIML):
    lists=[]
    vals=[]
    for x in range(len(sentence)): # tqdm(range(len(sentence))):
        arr=[]
        s=sentence[x]
        word = s.split() # splits string into a list
        length=len(word)
        sl=0 # should I make these start at 1?
        for i in (range(length)):
            sm=0
            token_l=nlp(word[i])
            for j in (range(length)):
                if(i!=j):
                    token_m=nlp(word[j])
                    sm=sm+token_l.similarity(token_m)
            sm=sm/length
            arr.append(sm)
            sm=abs(sm-WSIML)
            sl=sl+sm
        lists.append(arr)
        vals.append(size/sl)
    dqic4_df_lists=pd.DataFrame(lists) # df=pd.DataFrame(lists)
    dqic4_df_lists.to_csv("dqic4_df_lists.csv")
    # df.to_csv("path_to_wordsimgsabs/wordsimgabs.csv") # how do I direct .csvs?
    dqic4_df_vals=pd.DataFrame(vals) # df=pd.DataFrame(vals)
    dqic4_df_vals.to_csv("dqic4_df_vals.csv")
    # df.to_csv("path_to_dqic4gabs/dqic4gabs.csv") # ?

In [3]:
size=len(gwords)-1
WSIML=0.5
sentence=gwords['documents'] # gwords['fullnopunc'] # code requires only one column
parameter4(sentence,size,WSIML)

<ipython-input-2-1c430fdae79a>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "parameter4" failed type inference due to: Untyped global name 'nlp': Cannot determine Numba type of <class 'spacy.lang.en.English'>

File "<ipython-input-2-1c430fdae79a>", line 15:
def parameter4(sentence,size,WSIML):
    <source elided>
            sm=0
            token_l=nlp(word[i])
            ^

  @jit
<ipython-input-2-1c430fdae79a>:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "parameter4" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-2-1c430fdae79a>", line 7:
def parameter4(sentence,size,WSIML):
    <source elided>
    vals=[]
    for x in range(len(sentence)): # tqdm(range(len(sentence))):
    ^

  @jit
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "para

## How should the DQI c4 results be interpreted?

The numbers in each row of `dqic4_df_lists` are the similarity scores comparing two differents word in the same sentence. The scores in the single column of `dqic4_df_vals` are the averages across their corresponding rows in `dqic4_df_lists`. 

In his paper, Mishra categorizes values for `dqic4_df_vals` at or above 0.000372 as "good" and values at or below 0.000062 as "bad."